# Welcome

> meow Introduction and Quick Start

In [ ]:
from typing import List

import meow as mw
import numpy as np
from meow import Structure

## Installation

```sh
pip install meow-sim
```

## Quick Start

### 1. Structure
First create the structures to be simulated. A {class}`~.meow.structures.Structure` is a collection of a {class}`~.meow.geometries.Geometry` with a {class}`~.meow.materials.Material`. 

:::{note}
`meow` expects the propagation direction to be the `z`-axis! This makes the `zx`-plane parallel with the chip and the `y`-axis perpendicular to the chip. Keep this in mind when creating your structures.
:::

In [ ]:
length = 10.0

oxide = mw.Structure(
    material=mw.silicon_oxide,
    geometry=mw.Box(
        x_min=-1.0,
        x_max=1.0,
        y_min=-1.0,
        y_max=0.0,
        z_min=0.0,
        z_max=length,
    ),
)

core = mw.Structure(
    material=mw.silicon,
    geometry=mw.Box(
        x_min=-0.45/2,
        x_max=0.45/2,
        y_min=0.0,
        y_max=0.22,
        z_min=0,
        z_max=length,
    ),
)

structures = [oxide, core]
mw.visualize(structures)

:::{note} 
you can also extrude structures from a gds file. See {class}`~.meow.gds_structures.GdsExtrusionRule`. 
:::

### 2. Cells

Once you have the {class}`~.meow.structures.Structures` (a list that contains {class}`~.meow.structures.Structure` objects), they need to be divided into {class}`~.meow.cell.Cells`. A {class}`~.meow.cell.Cell` is a combination of those structures with 2D meshing info ({class}`~.meow.mesh.Mesh2d`) and a cell length.

In [ ]:
num_cells = 5

cells = mw.Cells(
    structures=structures,
    mesh=mw.Mesh2d(
        x = np.linspace(-1, 1, 101),
        y = np.linspace(-1, 1, 101),
    ),
    Ls=[length / num_cells for _ in range(num_cells)],
)

A cell should be a region of (approximate) constant cross section. We can use the {func}`~.meow.visualize.visualize` function to show the cross section of a cell:

In [ ]:
mw.visualize(cells[0])

### 3. Cross Sections
A {class}`~.meow.cell.Cell` contains all the {class}`~.meow.structures.Structure` info, but does not take any {class}`~.meow.environment.Environment` information (such as temparature or wavelength) into account. This information is important as it influences the refractive index of the {class}`~.meow.cross_section.CrossSection`.

Therefore, combining a {class}`~.meow.cell.Cell` with an {class}`~.meow.environment.Environment` yields a {class}`~.meow.cross_section.CrossSection`:

In [ ]:
env = mw.Environment(wl=1.55, T=25.0)
css = mw.CrossSections(cells, env)
mw.visualize(css[0])

### 4. Find Modes (FDE)
We can now compute multiple {class}`~.meow.modes.Modes` per {class}`~.meow.cross_section.CrossSection` using {func}`~.meow.fde.tidy3d.compute_modes`:

In [ ]:
num_modes = 2

modes: List[List[mw.Mode]] = []
for cs in css:
    modes_in_cs = mw.compute_modes(cs, num_modes=num_modes)
    modes.append(modes_in_cs)
    
# show Hx component of the second mode (idx 1) 
# of the first cell (idx 0):
mw.visualize(modes[0][1], fields=["Hx"])

:::{note} 
above, the {class}`~.meow.modes.Modes` of the {class}`~.meow.cross_section.CrossSection` are calculated sequentially. However, you're invited to try calculating the modes concurrently as well 😉
:::

### 5. Calculate S-matrix (EME)

The S-matrix of a collection of modes can now easily be calculated with {func}`~.meow.eme.tidy3d.compute_s_matrix`. This step uses the [sax](https://github.com/flaport/sax) circuit solver under the hood.

In [ ]:
S, port_map = mw.compute_s_matrix(modes)

print(port_map)
mw.visualize(S)

That's it! this was a quick introduction to the `meow` library. You're invited to check out the [examples](/shared/modeling/meow/gds_taper) or [to learn the ins and outs of the library](/shared/modeling/meow/library). Moreover, you're invited to contribute your own examples and code to the `meow` library as well!